In [14]:
import httpx
import time
import pandas as pd
import re
from transformers import pipeline

In [15]:
## Functions I might need later:

# def dedupe(xs):
#     return list(dict.fromkeys([x for x in xs if x]))

# env_post_ids = dedupe(env_post_ids)
# infr_post_ids = dedupe(infr_post_ids)
# housing_post_ids = dedupe(housing_post_ids)
# econ_post_ids = dedupe(econ_post_ids)
# life_qual_post_ids = dedupe(life_qual_post_ids)
# aesth_post_ids = dedupe(aesth_post_ids)
# gov_post_ids = dedupe(gov_post_ids)
# not_useful_post_ids = dedupe(not_useful_post_ids)

In [16]:
#MAX_POSTS = 500

all_post_ids = []

HEADERS = {
    "User-Agent": "research-script/0.1 (academic, non-commercial)"
}

subreddit = "nova"
search_url = f"https://www.reddit.com/r/{subreddit}/search.json"

params = {
    "q": 'datacenter',
    "restrict_sr": 1,
    "cId": "15d87a39-3f1c-4d17-8fbe-39ba6d1dc63e",
    "iId": "57f26489-620f-40bc-af0a-e3e5d57d1724",
}

env_post_ids = []
infr_post_ids = []
housing_post_ids = []
econ_post_ids = []
life_qual_post_ids = []
aesth_post_ids = []
gov_post_ids = []
not_useful_post_ids = []

after = None

environmental_keywords = ["environment", "pollution", "emissions", "diesel", "fumes", "light pollution", "water", "drought", "water consumption",  "waste", "carbon", "climate"]
infrastructure_utilities_keywords = ["utilities", "utility", "electric", "power", "grid", "blackout", "substation", "reliable", "reliability", "capacity", "infrastructure", "generator", "water"]
housing_keywords = ["housing", "house", "home", "rent", "property", "real estate", "house price", "housing price", "housing cost", "land cost", "land price", "gentrification"]
economic_keywords = ["job", "work", "revenue", "tax", "money", "rich", "salary", "economy", "investment", "business"]
life_quality_keywords = ["resident", "fumes", "noise", "loud", "vibration", "light pollution", "sound", "smell", "traffic"]
aesthetics_keywords = ["landscape", "visual impact",  "aesthetics", "looks", "architecture", "design", "facade", "industrial", "windows", "dark"]
government_keywords = ["zoning", "planning", "approval", "permit", "city council", "bill", "mayor", "governor", "government"]

# NEED TO INCLUDE THIS FILTERING
# dont_include = ["server", "rack", "switch", "firewall", "BGP", "latency"]

matched_posts = 0

with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
    while True:
        if after is not None:
            params["after"] = after
        else:
            params.pop("after", None)

        r = client.get(search_url, params=params)
        print("REQUEST URL:", str(r.url))
        r.raise_for_status()
        listing = r.json()

        children = listing["data"]["children"]
        if not children:
            break

        for child in children:
            # data = child.get("data", {})
            # post_id = data.get("id")

            # title = (data.get("title") or "")
            # selftext = (data.get("selftext") or "")
            # text = (title + "\n" + selftext).lower()

            post_id = child["data"]["id"]
            all_post_ids.append(post_id)
            title = (child["data"]["title"] or "")
            selftext = (child["data"]["selftext"] or "")
            text = (title + " " + selftext).lower()

            matched = False
        
            if any(kw in text for kw in environmental_keywords):
                env_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in infrastructure_utilities_keywords):
                infr_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in housing_keywords):
                housing_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in economic_keywords):
                econ_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in life_quality_keywords):
                life_qual_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in aesthetics_keywords):
                aesth_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in government_keywords):
                gov_post_ids.append(post_id)
                matched = True
            if not matched:
                not_useful_post_ids.append(post_id)
            else:
                matched_posts += 1
            
        after = listing["data"]["after"]
        print("After:", after)
        if after is None:
            break

        # if len(all_post_ids) >= MAX_POSTS:
        #     break

        time.sleep(0.2)

print("Total posts scanned:", len(all_post_ids))
print("Total posts with a theme:", matched_posts)
print("Found environmental posts:", len(env_post_ids))
print("Found infrastructure posts:", len(infr_post_ids))
print("Found housing posts:", len(housing_post_ids))
print("Found economic posts:", len(econ_post_ids))
print("Found life quality posts:", len(life_qual_post_ids))
print("Found aesthetic posts:", len(aesth_post_ids))
print("Found government posts:", len(gov_post_ids))
print(not_useful_post_ids)

REQUEST URL: https://www.reddit.com/r/nova/search.json?q=datacenter&restrict_sr=1&cId=15d87a39-3f1c-4d17-8fbe-39ba6d1dc63e&iId=57f26489-620f-40bc-af0a-e3e5d57d1724
After: t3_esy3zu
REQUEST URL: https://www.reddit.com/r/nova/search.json?q=datacenter&restrict_sr=1&cId=15d87a39-3f1c-4d17-8fbe-39ba6d1dc63e&iId=57f26489-620f-40bc-af0a-e3e5d57d1724&after=t3_esy3zu
After: t3_j25zf4
REQUEST URL: https://www.reddit.com/r/nova/search.json?q=datacenter&restrict_sr=1&cId=15d87a39-3f1c-4d17-8fbe-39ba6d1dc63e&iId=57f26489-620f-40bc-af0a-e3e5d57d1724&after=t3_j25zf4
After: None
Total posts scanned: 51
Total posts with a theme: 32
Found environmental posts: 3
Found infrastructure posts: 8
Found housing posts: 12
Found economic posts: 14
Found life quality posts: 11
Found aesthetic posts: 2
Found government posts: 8
['1p21o4w', '1244196', 'yylj6f', '1lg6mkl', '1pc87qw', '1abhhob', '1kylm7v', '1ki7msg', 'sp9ms4', 'esy3zu', '1kq8wz2', 'u7i038', 'p9y987', 'jrysr5', '1dxrg5p', '1fdrsh8', '2rv3ls', '67nayl'

After categorizing for themes, only 32 out of the 51 retreieved posts remain. However, I believe that this is a good thing because looking through the posts, none of them are revelant. With the other method of categorizing themes, all posts would be used, which I don't believe is good in this scenario.

In [17]:
nova_theme_lists = [env_post_ids, infr_post_ids, housing_post_ids, econ_post_ids, life_qual_post_ids, aesth_post_ids, gov_post_ids]
posts = pd.DataFrame(columns=["ids", "text", "date", "votes", "number of comments", "environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"])

for theme in nova_theme_lists:
    df1 = pd.DataFrame(columns=["ids", "text", "date", "votes", "number of comments", "environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"])
    post_ids = []
    post_texts = []
    post_dates = []
    post_votes = []
    # What is upvote ratio because for posts with 0 ups and 0 downs I've seen 0.40 something and then for posts with 100 ups and 0 downs ive seen 0.9
    # Should I use the upvotes and downvotes separately or should I use score (upvotes-downvotes) (I'll just use score because it feels like most of the time downs is 0 and I don't know if that's correct)
    post_comment_numbers = []
    with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
        for pid in theme:
            post_url = f"https://www.reddit.com/comments/{pid}.json"                
            r = client.get(post_url)
            r.raise_for_status()
            post_json = r.json()
            post_ids.append(pid)
            text_and_title = post_json[0]["data"]["children"][0]["data"]["title"] + " " + post_json[0]["data"]["children"][0]["data"]["selftext"]
            text_and_title = text_and_title.replace('\n', ' ')
            post_texts.append(text_and_title)
            post_dates.append(post_json[0]["data"]["children"][0]["data"]["created_utc"])
            post_votes.append(post_json[0]["data"]["children"][0]["data"]["score"])
            post_comment_numbers.append(post_json[0]["data"]["children"][0]["data"]["num_comments"])
    df1["ids"] = post_ids
    df1["text"] = post_texts
    df1["date"] = post_dates
    df1["votes"] = post_votes
    df1["number of comments"] = post_comment_numbers

    if theme == env_post_ids:
        df1["environment"] = True
    if theme == infr_post_ids:
        df1["infrastructure"] = True
    if theme == housing_post_ids:
        df1["housing"] = True
    if theme == econ_post_ids:
        df1["economy"] = True
    if theme == life_qual_post_ids:
        df1["life quality"] = True
    if theme == aesth_post_ids:
        df1["aesthetics"] = True
    if theme == gov_post_ids:
        df1["government"] = True
    posts = pd.concat([posts, df1], ignore_index=True)

/var/folders/9m/h28gbbc970j03ncf7v7dhqk80000gq/T/ipykernel_12445/1146076866.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  posts = pd.concat([posts, df1], ignore_index=True)


In [18]:
def remove_links(text):
    # Regex pattern to find typical URLs (http/https followed by non-whitespace characters)
    url_pattern = re.compile(r'\[https?://\S+\]|\(https?://\S+\)|\[www\.\S+\]|\(www\.\S+\)')
    # Replace found URLs with an empty string
    cleaned_text = url_pattern.sub('', text)
    return cleaned_text

sample_string = "The board’s land use policy committee heard a presentation from the Department of Planning and Development about potential enhancements to data center use standards, including requiring a noise study and establishing a minimum distance from residential areas. The board had directed staff last May to provide research, findings and recommendations...  McKay said this process is designed to steer data centers into places that are most appropriate and make sure standards are set for environmental protections, noise mitigation and other issues that have people concerned. He said the county is learning from Loudoun and Prince William Counties, where data centers are becoming more and more prevalent...  [https://wjla.com/news/local/fairfax-county-considers-enhancing-data-center-guidelines-virginia-community-development-board-of-supervisors#](https://wjla.com/news/local/fairfax-county-considers-enhancing-data-center-guidelines-virginia-community-development-board-of-supervisors#)  A group of residents from the [Save Bren Mar From Data Center](https://sites.google.com/view/savebrenmar/home) group from the Mason District held up signs throughout Tuesday's legislative policy committee meeting, calling calling on the Fairfax county Supervisors to end by right data center development on commercial and industrial properties.  [https://patch.com/virginia/reston/data-center-zoning-policy-be-updated-fairfax-county-board](https://patch.com/virginia/reston/data-center-zoning-policy-be-updated-fairfax-county-board)Fairfax County considers enhancing data center guidelines"
result = remove_links(sample_string)
print(result)

posts["text"] = posts["text"].apply(remove_links)

The board’s land use policy committee heard a presentation from the Department of Planning and Development about potential enhancements to data center use standards, including requiring a noise study and establishing a minimum distance from residential areas. The board had directed staff last May to provide research, findings and recommendations...  McKay said this process is designed to steer data centers into places that are most appropriate and make sure standards are set for environmental protections, noise mitigation and other issues that have people concerned. He said the county is learning from Loudoun and Prince William Counties, where data centers are becoming more and more prevalent...    A group of residents from the [Save Bren Mar From Data Center] group from the Mason District held up signs throughout Tuesday's legislative policy committee meeting, calling calling on the Fairfax county Supervisors to end by right data center development on commercial and industrial propert

In [19]:
grouping_cols = ["ids", "text", "date"]
posts = posts.groupby(grouping_cols, as_index=False).max()
theme_cols = ["environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"]
posts[theme_cols] = posts[theme_cols].fillna(False)
posts.head(30)

/var/folders/9m/h28gbbc970j03ncf7v7dhqk80000gq/T/ipykernel_12445/2437745036.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  posts[theme_cols] = posts[theme_cols].fillna(False)


,ids,text,date,votes,number of comments,environment,infrastructure,housing,economy,life quality,aesthetics,government
0,10s4xtb,People who have left NOVA or planning to leave...,1.675382e+09,129,308,False,False,False,False,False,False,True
1,117zom7,Is staying in nova even worth it? I posted thi...,1.676973e+09,8,22,False,False,True,True,False,False,True
2,1341j3h,Anyone have HD or 4k (drone) footage of Ashbur...,1.682885e+09,0,8,False,False,False,False,True,False,True
3,14gh0c1,Proposed data center in Chantilly stirs up con...,1.687473e+09,4,7,True,True,True,False,False,False,False
4,152m2sz,Microsoft acquires 14-acre site in Sterling fo...,1.689648e+09,87,43,False,False,True,False,True,False,False
5,158wywd,Can someone tell me whats going on on barriste...,1.690257e+09,0,7,False,False,True,False,True,False,False
6,15cwyr6,Aren't the Loudon datacenters actually awesome...,1.690649e+09,417,246,False,True,True,True,True,False,False
7,171nn12,"Looking for cable tech jobs Hello, I currentl...",1.696626e+09,0,4,False,False,True,True,False,False,False
8,1al34nd,Need help with sources and talking points AGAI...,1.707313e+09,0,27,False,False,True,False,False,False,False
9,1bdp55a,Fairfax County considers enhancing data center...,1.710329e+09,6,7,True,False,True,False,True,True,True


In [20]:
datacenters_keywords = ["datacenter", "data center", "datacentre", "data centre"]
datacenters_posts = 0

with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
        for pid in all_post_ids:
            post_url = f"https://www.reddit.com/comments/{pid}.json"                
            r = client.get(post_url)            
            post_json = r.json()
            text = post_json[0]["data"]["children"][0]["data"]["selftext"] + post_json[0]["data"]["children"][0]["data"]["title"]
            if any(kw in text.lower() for kw in datacenters_keywords):
                datacenters_posts += 1

print(datacenters_posts)

21


In [21]:
datacenters_posts = 0
datacenters_post_texts = []
a=0
invalid_posts = []

for text in posts["text"]:
    if any(kw in text.lower() for kw in datacenters_keywords):
        datacenters_posts += 1
        datacenters_post_texts.append(text)
    else:
        invalid_posts.append(posts.loc[a, "ids"])
    a += 1

print(datacenters_posts)
print(datacenters_post_texts)
print(invalid_posts)

17
["Anyone have HD or 4k (drone) footage of Ashburn? Hi, I'm a local youtuber planning on making a documentary on how NOVA (Loudoun and PWC) is the epicenter of the worlds datacenters.  Does anyone happen to have ground or drone footage of the data centers in Ashburn or elsewhere?   Thanks   The channel: ", 'Proposed data center in Chantilly stirs up controversy ahead of community meeting The applicant also wants a special exception to increase the allowed height of the data center from 75 feet to 110 feet, arguing that the added height is necessary to accommodate rooftop equipment and screening.  The data center option would include 50 parking spaces to the south of the three-story building, four loading spaces, and an 8-foot-tall security fence within the outside of an environmentally sensitive area.  The second option for a warehouse would include 150 parking spaces, a loading dock with 30 spaces, trailer parking, and a stormwater bio-retention area. The applicant would also pursue

Only 21 of the 51 posts that were in the reddit json when you search "datacenter" actually have the word datacenter in the main post, and only 17 out of the 32 that have a useful theme do. Based on the post ids I've looked at, this is because reddit marks a post as containing the search term if any one of the comments contains the search term. However, msny posts whose comments have the word datacenter in them don't actually talk about datacenters, but the comments might be helpful? (Removing links doesn't change the number, so it's all in the text).

Should I throw away these posts and only include posts that actually say datacenters, should I check the upvote ratio for the comments that include datacenters, or something else? I tried just starting with the nova subreddit and filtering from datacenters there instead of having reddit do it for me and it gave me 3 results out of the max 998 posts searched.

Update: When I expand the query to datacenter OR data center OR datacentre OR data centre, I get 131 results but only 3 of them actually have those words in the post, I've searched through some of the posts that come up and they don't even have the keywords in the comments.

In [22]:
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    truncation=True,      # <--- important
    padding=True,         # <--- for batching
    max_length=512        
)

Device set to use mps:0


In [23]:
texts = posts["text"].astype(str).tolist()
results = classifier(texts, truncation=True, padding=True, max_length=512, batch_size=32)
posts["roberta"] = results

In [24]:
posts["name"] = posts["roberta"].apply(lambda x: x["label"])

def roberta_label(name):
    if name == 'LABEL_2':
        return "positive"
    elif name == 'LABEL_1':
        return "neutral"
    else:
        return "negative"
posts["label"] = posts["name"].apply(roberta_label)

posts["degree"] = posts["roberta"].apply(lambda x: x["score"])
posts = posts.drop(["roberta", "name"], axis=1)
posts.head(15)

,ids,text,date,votes,number of comments,environment,infrastructure,housing,economy,life quality,aesthetics,government,label,degree
0,10s4xtb,People who have left NOVA or planning to leave...,1.675382e+09,129,308,False,False,False,False,False,False,True,neutral,0.752858
1,117zom7,Is staying in nova even worth it? I posted thi...,1.676973e+09,8,22,False,False,True,True,False,False,True,negative,0.797257
2,1341j3h,Anyone have HD or 4k (drone) footage of Ashbur...,1.682885e+09,0,8,False,False,False,False,True,False,True,neutral,0.837618
3,14gh0c1,Proposed data center in Chantilly stirs up con...,1.687473e+09,4,7,True,True,True,False,False,False,False,neutral,0.771397
4,152m2sz,Microsoft acquires 14-acre site in Sterling fo...,1.689648e+09,87,43,False,False,True,False,True,False,False,neutral,0.649346
5,158wywd,Can someone tell me whats going on on barriste...,1.690257e+09,0,7,False,False,True,False,True,False,False,positive,0.573035
6,15cwyr6,Aren't the Loudon datacenters actually awesome...,1.690649e+09,417,246,False,True,True,True,True,False,False,negative,0.503996
7,171nn12,"Looking for cable tech jobs Hello, I currentl...",1.696626e+09,0,4,False,False,True,True,False,False,False,negative,0.638340
8,1al34nd,Need help with sources and talking points AGAI...,1.707313e+09,0,27,False,False,True,False,False,False,False,negative,0.643528
9,1bdp55a,Fairfax County considers enhancing data center...,1.710329e+09,6,7,True,False,True,False,True,True,True,neutral,0.786951


In [25]:
# calculating average sentiment based on theme:
# Weigh all posts by their degree in the numerator and denominator, means that the average sentiment will just be +/- 1 if there are only positive or negative themes but other than that does a pretty good job of weighing neutrality
def avg_sentiment_calculation(theme):
    theme_posts = posts[posts[theme] == True]
    pos = theme_posts.loc[theme_posts["label"] == "positive", "degree"].sum()
    neg = theme_posts.loc[theme_posts["label"] == "negative", "degree"].sum()
    total = theme_posts["degree"].sum()
    return (pos-neg)/total

print("Average sentiment of environmental posts:", avg_sentiment_calculation("environment"))
print("Average sentiment of infrastructural posts:", avg_sentiment_calculation("infrastructure"))
print("Average sentiment of housing-related posts:", avg_sentiment_calculation("housing"))
print("Average sentiment of economic posts:", avg_sentiment_calculation("economy"))
print("Average sentiment of life-quality-related posts:", avg_sentiment_calculation("life quality"))
print("Average sentiment of aesthetics-related posts:", avg_sentiment_calculation("aesthetics"))
print("Average sentiment of governmental posts:", avg_sentiment_calculation("government"))

Average sentiment of environmental posts: 0.0
Average sentiment of infrastructural posts: -0.33685586099757103
Average sentiment of housing-related posts: -0.1676326067475207
Average sentiment of economic posts: -0.5120200545506511
Average sentiment of life-quality-related posts: -0.2697157448804726
Average sentiment of aesthetics-related posts: -0.5515159470014191
Average sentiment of governmental posts: -0.13496089631241784
